In [1]:
import open3d as o3d
import numpy as np

# Loading the point cloud
pcd = o3d.io.read_point_cloud("Point_clouds/armadillo.ply")

# Normal calculation
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

# Poisson Surface Reconstruction
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=9)

# Removal of noise based on density
vertices_to_remove = densities < np.quantile(densities, 0.01)
mesh.remove_vertices_by_mask(vertices_to_remove)

# Saving the reconstructed mesh
o3d.io.write_triangle_mesh("Point_clouds/reconstructed_mesh.ply", mesh)

# Visualization
o3d.visualization.draw_geometries([mesh])


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
# Loading the point cloud
pcd = o3d.io.read_point_cloud("Point_clouds/cow.ply")

# Normal calculation
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

# Poisson Surface Reconstruction
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=9)

# Removal of noise based on density
vertices_to_remove = densities < np.quantile(densities, 0.01)
mesh.remove_vertices_by_mask(vertices_to_remove)

# Saving the reconstructed mesh
o3d.io.write_triangle_mesh("Point_clouds/reconstructed_cow.ply", mesh)

# Visualization
o3d.visualization.draw_geometries([mesh])

In [4]:
'''

Ball Pivoting Algorithm BPA: generates triangle meshes from point clouds,
by “rolling” a ball over the point cloud and generating triangles
are generated when the ball hits three points.

'''


import open3d as o3d
import numpy as np

# Loading the point cloud
pcd = o3d.io.read_point_cloud("Point_clouds/cow.ply")

# Estimating the normals
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

# Ball pivoting algorithm
radii = [0.05, 0.1, 0.2]
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
    pcd, o3d.utility.DoubleVector(radii))

# Visualization
o3d.visualization.draw_geometries([mesh])

In [ ]:
'''
Alpha-Shapes: a method for generating surfaces from point clouds,
that can capture both concave and convex shapes.
'''
import open3d as o3d
import numpy as np

# Loading the point cloud
pcd = o3d.io.read_point_cloud("Point_clouds/cow.ply")

# Estimating the normals
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

# Set alpha value
alpha = 0.03

# Create Alpha Shape
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha)

# Calculate normals for the mesh
mesh.compute_vertex_normals()

# Visualization
o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)

# Experimenting with different alpha values
for alpha in np.logspace(np.log10(0.5), np.log10(0.01), num=4):
    print(f"Alpha = {alpha:.3f}")
    mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha)
    mesh.compute_vertex_normals()
    o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)

# Saving mesh
o3d.io.write_triangle_mesh("Point_clouds/alpha_shape_mesh.ply", mesh)


In [5]:
'''
Delaunay Triangulation: Open3D offers functions for Delaunay-
triangulation, which can be used to generate triangle meshes from point clouds.
point clouds.
'''

import open3d as o3d
import numpy as np
from scipy.spatial import Delaunay


pcd = o3d.io.read_point_cloud("Point_clouds/cow.ply")

# Converting the Open3D point cloud into a NumPy array
points = np.asarray(pcd.points)
points = points[:1000]

print(f"Number of points: {len(points)}")

unique_points = np.unique(points, axis=0)
print(f"Number of unique points: {len(unique_points)}")

# Performing the Delaunay triangulation
tri = Delaunay(points)

# Checking the Simplizes
if not all(len(simplex) == 4 for simplex in tri.simplices):
    print("Warning: Not all simplices have the length 4")

# Creating an Open3D mesh
mesh = o3d.geometry.TriangleMesh()
mesh.vertices = o3d.utility.Vector3dVector(points)

# Adding the triangles manually
triangle_list = []
for simplex in tri.simplices:
    if len(simplex) == 4:  # TDissect tetrahedrons into triangles
        triangle_list.extend([
            [simplex[0], simplex[1], simplex[2]],
            [simplex[0], simplex[1], simplex[3]],
            [simplex[0], simplex[2], simplex[3]],
            [simplex[1], simplex[2], simplex[3]]
        ])

# Converting the triangles into a NumPy array and adding them to the mesh
triangles = np.array(triangle_list, dtype=np.int32)
mesh.triangles = o3d.utility.Vector3iVector(triangles)

print(f"Number of triangles {len(mesh.triangles)}")

# Removing duplicate and invalid triangles
mesh.remove_duplicated_triangles()
mesh.remove_degenerate_triangles()
mesh.remove_unreferenced_vertices()

# Calculating the normals for the mesh
mesh.compute_vertex_normals()

print(f"Number of triangles after adjustment: {len(mesh.triangles)}")

# Visualization
o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)

# Saving mesh
o3d.io.write_triangle_mesh("Point_clouds/delaunay_mesh.ply", mesh)

Number of points: 1000
Number of unique points: 1000
Number of triangles 25548
Number of triangles after adjustment: 12878


True

In [6]:
'''
Load and visualize point cloud
'''
import open3d as o3d

# Laden einer PLY-Datei
pcd = o3d.io.read_point_cloud("Point_clouds/cow.ply")

# Visualisierung der Punktwolke
o3d.visualization.draw_geometries([pcd])

In [7]:
'''
Basic point cloud operations
************************************

The outlier removal method used in this code is called
“Statistical Outlier Removal” (statistical outlier removal).
The method 'remove_statistical_outlier()' in Open3D implements this algorithm.
algorithm.
1. for each point, the algo considers its k nearest neighbors (in this case
   in this case 20, defined by 'nb_neighbors=20')
2. for each point, the average distance to its k nearest neighbors is calculated. 
   neighbors is calculated.
3. then the mean and standard deviation of these average distances over all points is calculated.
   average distances over all points in the point cloud are calculated.
4. points whose average distance to their neighbors exceeds a
   number of standard deviations (in this case 2, defined by 'std_ratio
   by 'std_ratio=2.0') from the global mean value are considered
   are considered outliers and removed.
This approach is based on the assumption that outliers are typically
are typically further away from their neighbors than the majority of points in the
of the point cloud.
The static outlier removal is particularly useful for point clouds
with varying point density, as it takes local neighborhood information into 
instead of using global thresholds.
'''
# Downsampling of a point cloud
downpcd = pcd.voxel_down_sample(voxel_size=0.05)

# Estimate normal
downpcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

# Remove outliers
cl, ind = downpcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
inlier_cloud = downpcd.select_by_index(ind)

# Visualization of the point cloud
o3d.visualization.draw_geometries([inlier_cloud])

In [8]:
'''
customized registration
'''

import open3d as o3d
import numpy as np

def load_and_prepare_pointcloud(filename):
    pcd = o3d.io.read_point_cloud(filename)
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
    return pcd

def apply_noise_and_transform(pcd, noise_level=0.01, translation=(0.5, 0.5, 0)):
    noisy_pcd = o3d.geometry.PointCloud()
    noisy_pcd.points = o3d.utility.Vector3dVector(np.asarray(pcd.points).copy())
    points = np.asarray(noisy_pcd.points)
    noise = np.random.randn(points.shape[0], 3) * noise_level
    noisy_pcd.points = o3d.utility.Vector3dVector(points + noise)
    noisy_pcd.translate(translation)
    if pcd.has_normals():
        noisy_pcd.normals = o3d.utility.Vector3dVector(np.asarray(pcd.normals).copy())
    return noisy_pcd

def preprocess_point_cloud(pcd, voxel_size):
    pcd_down = pcd.voxel_down_sample(voxel_size)
    pcd_down.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 2, max_nn=30))
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 5, max_nn=100))
    return pcd_down, pcd_fpfh

def execute_global_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, True,
        distance_threshold,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(False), 3,
        [o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
         o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)],
        o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999))
    return result

def draw_registration_result(source, target, transformation):
    source_temp = o3d.geometry.PointCloud()
    source_temp.points = o3d.utility.Vector3dVector(np.asarray(source.points).copy())
    target_temp = o3d.geometry.PointCloud()
    target_temp.points = o3d.utility.Vector3dVector(np.asarray(target.points).copy())
    source_temp.paint_uniform_color([1, 0, 0])
    target_temp.paint_uniform_color([0, 1, 0])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

if __name__ == "__main__":
    source = load_and_prepare_pointcloud("Point_clouds/cow.ply")
    target = apply_noise_and_transform(source)

    voxel_size = 0.05
    source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
    target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)

    result_global = execute_global_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size)

    print("Global registration result:")
    print(result_global)
    draw_registration_result(source, target, result_global.transformation)

    current_transformation = result_global.transformation
    for i in range(5):
        threshold = 0.02 / (i + 1)
        reg_p2p = o3d.pipelines.registration.registration_icp(
            source, target, threshold, current_transformation,
            o3d.pipelines.registration.TransformationEstimationPointToPlane(),
            o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=100))
        current_transformation = reg_p2p.transformation
        print(f"ICP Iteration {i+1}:")
        print(reg_p2p)
        draw_registration_result(source, target, current_transformation)

    print("Final transformation:")
    print(current_transformation)
    draw_registration_result(source, target, current_transformation)

Global registration result:
RegistrationResult with fitness=1.000000e+00, inlier_rmse=2.262435e-02, and correspondence_set size of 2448
Access transformation to get result.
ICP Iteration 1:
RegistrationResult with fitness=7.406132e-01, inlier_rmse=1.337880e-02, and correspondence_set size of 2150
Access transformation to get result.
ICP Iteration 2:
RegistrationResult with fitness=1.956597e-01, inlier_rmse=7.435594e-03, and correspondence_set size of 568
Access transformation to get result.
ICP Iteration 3:
RegistrationResult with fitness=7.750603e-02, inlier_rmse=5.061217e-03, and correspondence_set size of 225
Access transformation to get result.
ICP Iteration 4:
RegistrationResult with fitness=3.479159e-02, inlier_rmse=3.735775e-03, and correspondence_set size of 101
Access transformation to get result.
ICP Iteration 5:
RegistrationResult with fitness=1.791250e-02, inlier_rmse=2.995822e-03, and correspondence_set size of 52
Access transformation to get result.
Final transformation:


In [9]:
'''
Advanced visualization
'''

def custom_draw_geometry(geometry):
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(geometry)
    
    # Customize render options
    opt = vis.get_render_option()
    opt.background_color = np.asarray([0.5, 0.5, 0.5])
    opt.point_size = 2.0
    
    # Set camera position
    ctr = vis.get_view_control()
    ctr.set_zoom(0.8)
    ctr.set_front([0, 0, -1])
    ctr.set_lookat([0, 0, 0])
    ctr.set_up([0, 1, 0])
    
    vis.run()
    vis.destroy_window()

custom_draw_geometry(pcd)

In [10]:
'''
Segmentation
'''

# Level segmentation
plane_model, inliers = pcd.segment_plane(distance_threshold=0.01, ransac_n=3, num_iterations=1000)

inlier_cloud = pcd.select_by_index(inliers)
outlier_cloud = pcd.select_by_index(inliers, invert=True)

o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

In [11]:
import open3d as o3d
import numpy as np

# Load your point cloud
pcd = o3d.io.read_point_cloud("Point_clouds/cow.ply")

# Check whether the point cloud has colors
if not pcd.has_colors():
    print("The point cloud has no colors. Add random colors.")
    # Add random colors
    pcd.colors = o3d.utility.Vector3dVector(np.random.uniform(0, 1, size=(len(pcd.points), 3)))

# Create the Octree
octree = o3d.geometry.Octree(max_depth=4)
octree.convert_from_point_cloud(pcd, size_expand=0.01)

# Visualize the Octree
o3d.visualization.draw_geometries([octree])

# Create the KDTree
pcd_tree = o3d.geometry.KDTreeFlann(pcd)

# Check whether the point cloud has enough points
if len(pcd.points) > 1000:
    # Search for k nearest neighbors
    [k, idx, _] = pcd_tree.search_knn_vector_3d(pcd.points[1000], min(200, len(pcd.points)))
    
    # Color the neighbors blue
    np.asarray(pcd.colors)[idx[1:], :] = [0, 0, 1]
else:
    print("The point cloud has less than 1000 points. Select a smaller index.")
    # Select a random point and its neighbors
    random_index = np.random.randint(0, len(pcd.points))
    [k, idx, _] = pcd_tree.search_knn_vector_3d(pcd.points[random_index], min(200, len(pcd.points)))
    np.asarray(pcd.colors)[idx[1:], :] = [0, 0, 1]

# Visualize the point cloud
o3d.visualization.draw_geometries([pcd])

The point cloud has no colors. Add random colors.
